In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('./train_final.csv')

### Литвинов Н.А.

In [2]:
cols = ['stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'total_of_special_requests']
df_2 = df[cols].copy()

In [3]:
df_2.head()

,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,total_of_special_requests
0,0,2,2,0.0,0,0
1,1,2,2,0.0,0,2
2,1,5,2,2.0,0,3
3,2,0,2,0.0,0,0
4,2,4,2,0.0,0,0


In [4]:
df_2.describe()

,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,total_of_special_requests
count,44638.000000,44638.000000,44638.000000,44638.000000,44638.000000,44638.000000
mean,0.931516,2.507303,1.852883,0.102424,0.008289,0.603253
std,1.002021,1.928111,0.578898,0.393273,0.091649,0.803659
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000
50%,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000
75%,2.000000,3.000000,2.000000,0.000000,0.000000,1.000000
max,16.000000,40.000000,55.000000,3.000000,2.000000,5.000000


In [5]:
df_2.isnull().sum()

stays_in_weekend_nights      0
stays_in_week_nights         0
adults                       0
children                     0
babies                       0
total_of_special_requests    0
dtype: int64

##### FEATURE ENGINEERING

In [10]:
# Сумма ночевок
df['total_nights'] = df_2['stays_in_weekend_nights'] + df_2['stays_in_week_nights']
df_2['total_nights'] = df_2['stays_in_weekend_nights'] + df_2['stays_in_week_nights']
# Проживание длительностью > 7 ночей
df['long_stay_flag'] = (df_2['total_nights'] > 7).astype(int)
df_2['long_stay_flag'] = (df_2['total_nights'] > 7).astype(int)
# Общее кол-во гостей
df['total_guests'] = df_2[['adults','children','babies']].sum(axis=1)
df_2['total_guests'] = df_2[['adults','children','babies']].sum(axis=1)
# Плотность проживания
df['guests_per_night'] = df_2['total_guests'] / np.where(df_2['total_nights'] == 0, np.nan, df_2['total_nights'])
df_2['guests_per_night'] = df_2['total_guests'] / np.where(df_2['total_nights'] == 0, np.nan, df_2['total_nights'])
# Кол-во запросов на гостя
df['special_requests_per_guest'] = df_2['total_of_special_requests'] / df_2['total_guests']
df_2['special_requests_per_guest'] = df_2['total_of_special_requests'] / df_2['total_guests']
# Имеется ли special_request
df['has_special_request'] = (df['total_of_special_requests'] > 0).astype(int)
df_2['has_special_request'] = (df_2['total_of_special_requests'] > 0).astype(int)

In [11]:
df_2.head()

,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,total_of_special_requests,total_nights,long_stay_flag,total_guests,guests_per_night,special_requests_per_guest,has_special_request
0,0,2,2,0.0,0,0,2,0,2.0,1.000000,0.00,0
1,1,2,2,0.0,0,2,3,0,2.0,0.666667,1.00,1
2,1,5,2,2.0,0,3,6,0,4.0,0.666667,0.75,1
3,2,0,2,0.0,0,0,2,0,2.0,1.000000,0.00,0
4,2,4,2,0.0,0,0,6,0,2.0,0.333333,0.00,0


In [12]:
df_2.isnull().sum()

stays_in_weekend_nights         0
stays_in_week_nights            0
adults                          0
children                        0
babies                          0
total_of_special_requests       0
total_nights                    0
long_stay_flag                  0
total_guests                    0
guests_per_night              290
special_requests_per_guest     49
has_special_request             0
dtype: int64

In [13]:
df.to_csv("train_final_processed_data.csv", index=False)